#### This basic tutorial shows you how to interface the TCS3425 color sensor with STM32 Nucleo-L432KC. The datasheet of the TCS3425 serises can be found [here](https://cdn-shop.adafruit.com/datasheets/TCS34725.pdf). This tutorial is developed based on the tutorial [here](https://www.micropeta.com/video114).

## 1. Wiring

<img src="Figures\STM_pinout.png" alt="STM32 Nucleo-L432KC Pinout" width="800"/>

<img src="Figures\TCS3472.jpg" alt="TCS3472" width="300"/>

The TCS3472 communicates with the STM32 using I<sup>2</sup>C. Connect:
- TCS3472 VIN <--> STM32 5V
- TCS3472 GND <--> STM32 GND
- TCS3472 SDA <--> STM32 PA_10 (I2C1_SDA)
- TCS3472 SCL <--> STM32 PA_9 (I2C1_SCL)

## 2. CubeIDE Pinout & Configuration

In STM32CubeIDE, navigate to the .ioc file of the project. Under the `Pinout & Configuration` tab, click on the `Connectivity` dropdown list. Click `I2C1`, select `I2C` for the mode. In the `Parameter Settings`, you can change the `I2C Speed Mode` to `Fast Mode`. After these settings, `PA10` and `PA9` in the `Pinout view` should turn green. (If different pins turn green in your case, you can use them instead of PA10 and PA9, just don't forget to change your hardware wiring.)

<img src="Figures\color_sensor_I2C_configuration.PNG" alt="I2C configuration" width="1000"/>

In order to debug, in the `System Core` dropdown list, click `SYS` and make sure the `Debug` mode is set to `Serial Wire`.

<img src="Figures\Debug_configuration.PNG" alt="Debug configuration" width="1000"/>

After these steps, save the file and generate code.

## 3. Coding

Open the main.c file. Copy the following code and paste between `/* USER CODE BEGIN 0 */` and `/* USER CODE END 0 */`.

```c
/* USER CODE BEGIN 0 */
#define TCS3472_ADDRESS          (0x29 << 1) /* I2C address */
#define TCS3472_COMMAND_BIT      (0x80)      /* Command bit */
#define TCS3472_ENABLE           (0x00)      /* Enable register */
#define TCS3472_ENABLE_AEN       (0x02)      /* RGBC Enable */
#define TCS3472_ENABLE_PON       (0x01)      /* Power on */
#define TCS3472_ATIME            (0x01)      /* Integration time */
#define TCS3472_CONTROL          (0x0F)      /* Set the gain level */
#define TCS3472_ID               (0x12)
/* The ID should be 0x44 for TCS34721/TCS34725, 0x4D for TCS34723/TCS34727 */
#define TCS3472_CDATAL           (0x14)      /* Clear channel data */
#define TCS3472_CDATAH           (0x15)
#define TCS3472_RDATAL           (0x16)      /* Red channel data */
#define TCS3472_RDATAH           (0x17)
#define TCS3472_GDATAL           (0x18)      /* Green channel data */
#define TCS3472_GDATAH           (0x19)
#define TCS3472_BDATAL           (0x1A)      /* Blue channel data */
#define TCS3472_BDATAH           (0x1B)
/* set the correct delay time in void getRawData() for TCS3472_INTEGRATIONTIME */
#define TCS3472_INTEGRATIONTIME_50MS   0xEB  /* 50ms  - 20 cycles */
#define TCS3472_GAIN_4X                0x01  /* 4x gain  */

uint8_t _TCS3472Initialised = 0;
int red, green, blue;

void write8 (uint8_t reg, uint32_t value);
uint8_t read8(uint8_t reg);
uint16_t read16(uint8_t reg);
void enable(void);
void disable(void);
void setIntegrationTime(uint8_t it);
void setGain(uint8_t gain);
void tcs3272_init( void );
void getRawData (uint16_t *r, uint16_t *g, uint16_t *b, uint16_t *c);
void getRGB(int *R, int *G, int *B);

/* Writes a register and an 8 bit value over I2C */
void write8 (uint8_t reg, uint32_t value)
{
    uint8_t txBuffer[2];
    txBuffer[0] = (TCS3472_COMMAND_BIT | reg);
    txBuffer[1] = (value & 0xFF);
    HAL_I2C_Master_Transmit(&hi2c1, TCS3472_ADDRESS, txBuffer, 2, 100);
}

/* Reads an 8 bit value over I2C */
uint8_t read8(uint8_t reg)
{
    uint8_t buffer[1];
    buffer[0] = (TCS3472_COMMAND_BIT | reg);
    HAL_I2C_Master_Transmit(&hi2c1, TCS3472_ADDRESS, buffer, 1, 100);
    HAL_I2C_Master_Receive(&hi2c1, TCS3472_ADDRESS, buffer, 1, 100);
    return buffer[0];
}

/* Reads a 16 bit values over I2C */
uint16_t read16(uint8_t reg)
{
  uint16_t ret;
    uint8_t txBuffer[1],rxBuffer[2];
    txBuffer[0] = (TCS3472_COMMAND_BIT | reg);
    HAL_I2C_Master_Transmit(&hi2c1, TCS3472_ADDRESS, txBuffer, 1, 100);
    HAL_I2C_Master_Receive(&hi2c1, TCS3472_ADDRESS, rxBuffer, 2, 100);
    ret = rxBuffer[1];
    ret <<= 8;
    ret |= rxBuffer[0] & 0xFF;
  return ret;
}

void enable(void)
{
  write8(TCS3472_ENABLE, TCS3472_ENABLE_PON);
  HAL_Delay(3);
  write8(TCS3472_ENABLE, TCS3472_ENABLE_PON | TCS3472_ENABLE_AEN);
  HAL_Delay(50);
}

void disable(void)
{
  /* Turn the device off to save power */
  uint8_t reg = 0;
  reg = read8(TCS3472_ENABLE);
  write8(TCS3472_ENABLE, reg & ~(TCS3472_ENABLE_PON | TCS3472_ENABLE_AEN));
}

void setIntegrationTime(uint8_t itime)
{
  if (_TCS3472Initialised == 0) tcs3272_init();
  write8(TCS3472_ATIME, itime);
}

void setGain(uint8_t gain)
{
  if (_TCS3472Initialised == 0) tcs3272_init();
  write8(TCS3472_CONTROL, gain);
}

void tcs3272_init(void)
{
  /* Make sure we're actually connected */
  uint8_t readValue = read8(TCS3472_ID);
  if ((readValue != 0x44) && (readValue != 0x4D))
  {
    return;
  }
  _TCS3472Initialised = 1;
  /* Set default integration time and gain */
  setIntegrationTime(TCS3472_INTEGRATIONTIME_50MS);
  setGain(TCS3472_GAIN_4X);
  /* Note: by default, the device is in power down mode on bootup */
  enable();
}

/* Get raw data */
void getRawData (uint16_t *r, uint16_t *g, uint16_t *b, uint16_t *c)
{
  if (_TCS3472Initialised == 0) tcs3272_init();

  *c = read16(TCS3472_CDATAL);
  *r = read16(TCS3472_RDATAL);
  *g = read16(TCS3472_GDATAL);
  *b = read16(TCS3472_BDATAL);
  /* Delay time is from page no 16/26 from the datasheet  (256 − ATIME)* 2.4ms */
  HAL_Delay(50); // Set delay for (256 − 0xEB)* 2.4ms = 50ms
}

/* Get Red, Green and Blue color from Raw Data */
void getRGB(int *R, int *G, int *B)
{
    uint16_t rawRed, rawGreen, rawBlue, rawClear;
    getRawData(&rawRed, &rawGreen, &rawBlue, &rawClear);
    if(rawClear == 0)
    {
      *R = 0;
      *G = 0;
      *B = 0;
    }
    else
    {
      *R = (int)rawRed * 255 / rawClear;
      *G = (int)rawGreen * 255 / rawClear;
      *B = (int)rawBlue * 255 / rawClear;
    }
}
/* USER CODE END 0 */
```

Copy the following code and paste between `/* USER CODE BEGIN WHILE */` and `/* USER CODE END WHILE */`:

```c
  /* USER CODE BEGIN WHILE */
  while (1)
  {
	getRGB(&red, &green, &blue);
	HAL_Delay(500);
    /* USER CODE END WHILE */
```

## 4. Debugging

Now, click on `Debug` and make sure the `Enable live expressions` option in the Debugger configuration is selected, which lets us see the changes of the color values.

<img src="Figures\Debugger_configuration.png" alt="Debugger configuration" width="700"/>

After entering the debugging mode, add `red`, `green` and `blue` under the `Live Expressions` tab. These values are the RGB values of the color detected by the color sensor, ranging between 0 and 255.

<img src="Figures\Live_exp.png" alt="Live expression" width="1000"/>

Now click `Resume` to run the code, and see the values change when you place difference color in front of the sensor.

<img src="Figures\Color_change.png" alt="Color change" width="1000"/>

## 5. Additional functions

You can change the `integration time` and the `gain` by modifying these two parameters before running the code:

```c
#define TCS34725_INTEGRATIONTIME_50MS   0xEB  /* 50ms  - 20 cycles */
#define TCS34725_GAIN_4X                0x01  /* 4x gain  */
```

Or call the `setIntegrationTime()` and `setGain()` functions at appropriate time.